In [ ]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
from utility_functions import random_imputation

In [ ]:
# load data
data = pd.read_csv('ESS8 data/ESS8_cleaned_wmissingvals.csv', low_memory=False)

In [ ]:
# make df with missingness percentage of features with missingness
missing_cols = data.columns[data.isnull().any()].to_list()
percent_missing = data[missing_cols].isnull().sum() * 100 / len(data)
missing_info = pd.DataFrame({'column name':missing_cols,
                             'percentage missing':percent_missing})

# extract rows with >= 70% missing
many_missing = missing_info[missing_info['percentage missing'] > 70]
many_missing

Alternative to standard correlation heatmap: https://towardsdatascience.com/better-heatmaps-and-correlation-matrix-plots-in-python-41445d0f2bec

In [ ]:
## remove features with >70% missing and with one value in entire data set
many_missing_feat = many_missing.index.tolist()

df_cleaned_missing = data.drop(columns=many_missing_feat)
for col in df_cleaned_missing.columns:
    if len(df_cleaned_missing[col].unique()) == 1:
        df_cleaned_missing.drop(col, inplace=True, axis=1)
nr_feat = len(df_cleaned_missing.columns)
print(f"Number of features after removing >70% missing and 1-value features: {nr_feat}")

In [ ]:
# check cleaning results
df_cleaned_missing.nunique()

In [ ]:
# check cleaning results
df_cleaned_missing.info()

In [ ]:
v = df_cleaned_missing.idno.value_counts()
df_cleaned_missing[df_cleaned_missing.idno.isin(v.index[v.gt(2)])]

In [ ]:
# ID number has identical values, but this does not look informative and hence is removed
print(df_cleaned_missing[df_cleaned_missing['idno'] == 1304])

# remove idno column from data
df_cleaned_missing = df_cleaned_missing.drop(columns=['idno'])

In [ ]:
# check what and how many other years are present in the data
df_cleaned_missing[df_cleaned_missing['inwyye'] != 2016.0]

In [ ]:
# save new data set
df_cleaned_missing.to_csv('ESS8 data/ESS8_subset_cleaned_wmissingvals.csv', index=False)

In [ ]:
# encode object type data
enc = OrdinalEncoder()
data_obj = df_cleaned_missing.select_dtypes(include=object)
enc.fit(data_obj)
encoding = enc.fit_transform(data[data_obj.columns])

c = 0

for i in data_obj.columns:
    df_cleaned_missing[i] = encoding[:, c]
    c += 1

df_cleaned_missing.info()

In [ ]:
pd.set_option('display.max_columns', None)
df_cleaned_missing[df_cleaned_missing[['inwtm']].isnull().any(axis=1)]


In [ ]:
# if the end year is missing then there is a start year and vice versa, so fill with each other
df_cleaned_missing['inwyye'] = df_cleaned_missing['inwyye'].fillna(df_cleaned_missing['inwyys'])
df_cleaned_missing['inwyys'] = df_cleaned_missing['inwyys'].fillna(df_cleaned_missing['inwyye'])

In [ ]:
# replace 0 with 24 and calculate total time
df_cleaned_missing['inwshh'].replace(0, 24, inplace=True)
df_cleaned_missing['inwehh'].replace(0, 24, inplace=True)
df_cleaned_missing[df_cleaned_missing[['inwtm']].isnull().any(axis=1)]

In [ ]:
# this could be done once after random imputation but keep it here to check percentage/nr rows still missing
df_cleaned_missing['inwtm'] = df_cleaned_missing['inwtm'].fillna(abs((df_cleaned_missing['inwehh'].mul(60) +
                                                                  df_cleaned_missing['inwemm']) -
                                                                 (df_cleaned_missing['inwshh'].mul(60) +
                                                                  df_cleaned_missing['inwsmm'])))

df_cleaned_missing[df_cleaned_missing[['inwtm']].isnull().any(axis=1)]

In [ ]:
# random imputation
df_cleaned_missing['inwtm'] = random_imputation(df_cleaned_missing['inwtm'].to_frame())

df_cleaned_missing['inwtm'].describe()

In [ ]:
#no missing values in inwtm anymore :)
df_cleaned_missing[df_cleaned_missing[['inwtm']].isnull().any(axis=1)]

In [ ]:
# drop other interview time data (so except year and month)
df_cleaned_missing.drop(['inwemm', 'inwehh', 'inwsmm', 'inwshh'], axis=1, inplace=True) #remove columns with all identical vals
# df_cleaned_missing.shape()

In [ ]:
# check how many missing months and days
missing_endmo = len(df_cleaned_missing[df_cleaned_missing[['inwmme']].isnull().any(axis=1)])
missing_enddy = len(df_cleaned_missing[df_cleaned_missing[['inwdde']].isnull().any(axis=1)])
missing_stamo = len(df_cleaned_missing[df_cleaned_missing[['inwmms']].isnull().any(axis=1)])
missing_stady = len(df_cleaned_missing[df_cleaned_missing[['inwdds']].isnull().any(axis=1)])

print(f"""nr missing month end: {missing_endmo}\n
nr missing day end: {missing_enddy}\n
nr missing month start: {missing_stamo}\n
nr missing day start: {missing_stady}""")

In [ ]:
# not many missing in month and day columns, so use random imputation
df_cleaned_missing['inwmme'] = random_imputation(df_cleaned_missing['inwmme'].to_frame())
df_cleaned_missing['inwdde'] = random_imputation(df_cleaned_missing['inwdde'].to_frame())
df_cleaned_missing['inwmms'] = random_imputation(df_cleaned_missing['inwmms'].to_frame())
df_cleaned_missing['inwdds'] = random_imputation(df_cleaned_missing['inwdds'].to_frame())

In [ ]:
# check if numerical features have missingness; they do !!!
df_cleaned_missing[df_cleaned_missing[['dweight', 'pspwght','pweight', 'anweight', 'nwspol',
            'netustm','agea', 'eduyrs', 'wkhct', 'wkhtot',
            'wkhtotp', 'inwtm']].isnull().any(axis=1)]

In [ ]:
# save cleaned data to csv file
df_cleaned_missing.to_csv('ESS8 data/ESS8_subset_cleaned_timeadj_wmissingvals.csv', index=False)

Below, some initial checks are done to gain insight into what data sets will be constructed
during the processing pipeline.

In [ ]:
## check if loading the data works and extract all targets (i.e. >5% missing)

# load data
data3 = pd.read_csv('ESS8 data/ESS8_subset_cleaned_timeadj_wmissingvals.csv', low_memory=False)

# make df with missingness percentage of features with missingness
missing_cols = data3.columns[data3.isnull().any()].to_list()
percent_missing = data3[missing_cols].isnull().sum() * 100 / len(data3)
missing_info = pd.DataFrame({'column name':missing_cols,
                             'percentage missing':percent_missing})

# extract rows with > 5% missing (74 features)
many_missing = missing_info[missing_info['percentage missing'] > 5]
# print(len(many_missing))

targets3 = many_missing['column name'].tolist()
targets3

